In [1]:
import sys
sys.path.append('/Users/avgalichin/Documents/kaggle/Jigsaw2022')
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer

from src.data_utils import (clean, encode_df)

#### Download and clean data

In [ ]:
def clean(df: pd.DataFrame, col_name: str = 'text', min_len: int = 15):
    df.loc[:, col_name] = df[col_name].str.replace('https?://\S+|www\.\S+', ' social medium ')
    df.loc[:, col_name] = df[col_name].str.replace('\s+', ' ')
    df.loc[:, col_name] = df[col_name].str.strip()
    df = df[df[col_name].str.len() > min_len]

    df.reset_index(drop=True, inplace=True)

    return df

In [8]:
main_dir = Path('/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets')
data_dir = main_dir / 'Initial'

In [ ]:
data = pd.read_csv(data_dir / 'jigsaw22_validation_data.csv')
print(data.shape)
data.head()

In [ ]:
data = clean(data.copy(), col_name='less_toxic', min_len=0)
data = clean(data.copy(), col_name='more_toxic', min_len=0)
data = data.drop_duplicates(subset=['less_toxic', 'more_toxic'])
data.reset_index(drop=True, inplace=True)

print(data.shape)
data.head()

In [9]:
from src.models import _initialize_hugginface_pair_model
from src.dataset import Jigsaw22DatasetPaired

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [11]:
data = Jigsaw22DatasetPaired(
    [data_dir / 'jigsaw22_validation_data.csv'],
    tokenizer,
    batch_size=2,
    shuffle=False)

/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:176: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('https?://\S+|www\.\S+', ' social medium ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:177: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('\s+', ' ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/data_utils.py:89: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)


In [12]:
for ex in data.dataset.take(1):
    pass

In [ ]:
model = _initialize_hugginface_pair_model(n_classes=1)

In [14]:
output = model(ex[0])

In [10]:
from src.losses import MarginRankingLoss
import tensorflow as tf

mrl = MarginRankingLoss(margin=0.5)

mrl(tf.cast(ex[1], tf.float32)[:, None], output)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>